# Model descriptions

The Mitochondrial Retrograde (RTG) Signalling Model is a system of 11 coupled ODEs.

In [ ]:
using RetroSignalModel
using ModelingToolkit

@named sys = RtgMTK(; simplify=false)


## Parameter estimation

Simulated annealing (SAMIN) with bounds was used to find a set of parameters that fit exprerimental conditions. The following code is just a demonstration. In peactice we ran the code for much longer.

[Source code](https://github.com/ntumitolab/RetroSignalModel.jl)

In [ ]:
using RetroSignalModel
using Optim

optim_params(targetratio=3, optimoptions=Optim.Options(iterations=100, show_trace=true, show_every=10))


## Valid parameter sets

In [ ]:
using RetroSignalModel
using CSV
using DataFrames
using Plots


In [ ]:
filename = joinpath(dirname(pathof(RetroSignalModel)), "data", "solution_rtgMTK_optim.csv")
dfoptim = let
    df = CSV.read(filename, DataFrame)
    df[!, Not(:n_S)] .= log10.(df[!, Not(:n_S)])
end

res = map(sort(names(dfoptim))) do lab
    histogram(dfoptim[!, lab], label=lab)
end

plot(res..., layout=(6, 4), size=(1024, 1024))
# savefig("optimparams.png")


In [ ]:
filename = joinpath(dirname(pathof(RetroSignalModel)), "data", "solution_rtgM4.csv")
dforig = let
    df = CSV.read(filename, DataFrame)
    df[!, Not(:n_S)] .= log10.(df[!, Not(:n_S)])
end

res = map(sort(names(dforig))) do lab
    histogram(dforig[!, lab], label=lab)
end

plot(res..., layout=(6, 4), size=(1024, 1024))
# savefig("randomparams.png")
